In [1]:
import cv2
import numpy as np
from keras.models import model_from_json, load_model

In [2]:
emotion_dict = {0:'Angry', 1:'Disgust', 2:'Fear', 3:'Happy', 4:'Neutral', 5:'Sad',6:'Surprise'}

In [3]:
# load json and create model
json_file = open('model/model_50.json','r')
loaded_model_json = json_file.read()
json_file.close()
emotion_model = model_from_json(loaded_model_json)

In [3]:
# load weights into model
# model = emotion_model.load_weights('model/model_50.weights.h5')
model = load_model('model/model_10.h5')
print('Loaded model from disk')



Loaded model from disk


In [4]:
cap = cv2.VideoCapture(0)

In [5]:
while True:
    ret, frame = cap.read()
    frame = cv2.resize(frame,(1280, 720))
    if not ret:
        break
    faceDetect = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # detect faces on camera
    num_faces = faceDetect.detectMultiScale(gray_frame, 1.3, 5)
    
    for (x, y, w, h) in num_faces:
        cv2.rectangle(frame, (x,y-50), (x+w,y+h+10), (0,255,0), 4)
        roi_gray_frame = gray_frame[y:y+h, x:x+w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame,(48,48)), -1),0)
        
        # prediction 
        emotion_prediction = model.predict(cropped_img)
        max_index = int(np.argmax(emotion_prediction))
        cv2.putText(frame, emotion_dict[max_index], (x+5, y-20), cv2.FONT_HERSHEY_SIMPLEX,1,(255, 0,0),2, cv2.LINE_4)
    cv2.imshow("Emotion: ", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cv2.release()
cv2.destroyAllWindows()


ValueError: in user code:

    File "C:\Users\91810\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 2440, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\91810\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 2425, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\91810\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 2413, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\91810\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 2381, in predict_step
        return self(x, training=False)
    File "C:\Users\91810\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from Non

In [6]:
while True:
    ret, frame = cap.read()
    frame = cv2.resize(frame, (1280, 720))  # Resize frame if needed

    if not ret:
        break

    # Convert frame to grayscale for face detection
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces using cascade classifier
    num_faces = faceDetect.detectMultiScale(gray_frame, 1.3, 5)

    for (x, y, w, h) in num_faces:
        # Draw rectangle around detected faces
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (0, 255, 0), 4)

        # Extract the region of interest (ROI) for the face
        roi_gray_frame = gray_frame[y:y+h, x:x+w]

        # Preprocess the ROI for model prediction:
        # 1. Resize to match MobileNetV2 input size
        roi_resized = cv2.resize(roi_gray_frame, (48, 48))

        # 2. Add two extra channels filled with zeros
        roi_color = np.expand_dims(roi_resized, axis=-1)  # Expand to add a single channel
        roi_color = np.repeat(roi_color, 3, axis=-1)  # Repeat 3 times to create 3 channels

        # 3. Normalize (optional, might have been done during training)
        roi_color = roi_color / 255.0

        # 4. Reshape to match expected input shape
        cropped_img = roi_color.reshape(1, 48, 48, 3)

        # Make prediction using the model
        emotion_prediction = model.predict(cropped_img)
        max_index = int(np.argmax(emotion_prediction))

        # Display predicted emotion label on the frame
        cv2.putText(frame, emotion_dict[max_index], (x+5, y-20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_4)

    # Display the frame with detected faces and emotions
    cv2.imshow("Emotion Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 80ms/step
